In [66]:
import sys
print(sys.executable)

/opt/conda/bin/python


In [67]:
%pip install datasets
%pip install transformers
%pip install datasets
import numpy
import transformers
import datasets


print(numpy.__version__)
print(transformers.__version__)
print(datasets.__version__)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
2.1.2
4.57.3
4.4.1


In [68]:
%pip install 'accelerate>=0.26.0'

Note: you may need to restart the kernel to use updated packages.


In [69]:

from datasets import load_dataset

In [70]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("e9t/nsmc", revision="refs/convert/parquet")

# split train into train and validation
split = dataset["train"].train_test_split(test_size=0.1, seed=42)

# wrap into a proper DatasetDict
hf_dataset = DatasetDict({
    "train": split["train"],
    "validation": split["test"],
    "test": dataset["test"]
})

In [71]:
cols = hf_dataset["train"].column_names   
cols





['id', 'document', 'label']

In [72]:
for i in range(5):
    for col in cols:
        print(f"{col}: {hf_dataset['train'][col][i]}")
    print("-----")

id: 5960467
document: 명작은 옛날과 현재의 동일한 공감대 위에 놓인다. 그리고 더스틴 호프만의 섹시미^^
label: 1
-----
id: 5244343
document: 원작에 미원을 치다.
label: 0
-----
id: 8786278
document: 경상도로 하고싶었지만 티날까봐강원도로 하셨나? 가만있는 강원도만불쌍하게 된겨?
label: 0
-----
id: 5452084
document: 나래이션 미쳤다 KBS 나레이션은 진짜 멋졌다!! 극장나래이션은 별점이아깝다!
label: 0
-----
id: 10045037
document: 영화 자체도 좋았지만. 아오이유우 때문에 모에사 할뻔했다.
label: 1
-----


In [73]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "klue/bert-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

"""
 
# 모든 BERT 레이어 동결
for param in model.bert.parameters():
    param.requires_grad = False

# 상위 6개 레이어 unfreeze
num_layers_to_finetune = 6
total_layers = 12

for i in range(total_layers - num_layers_to_finetune, total_layers):
    for param in model.bert.encoder.layer[i].parameters():
        param.requires_grad = True
    print(f"Layer {i} trainable")

# 분류기 항상 학습
for param in model.classifier.parameters():
    param.requires_grad = True

"""

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'\n \n# 모든 BERT 레이어 동결\nfor param in model.bert.parameters():\n    param.requires_grad = False\n\n# 상위 6개 레이어 unfreeze\nnum_layers_to_finetune = 6\ntotal_layers = 12\n\nfor i in range(total_layers - num_layers_to_finetune, total_layers):\n    for param in model.bert.encoder.layer[i].parameters():\n        param.requires_grad = True\n    print(f"Layer {i} trainable")\n\n# 분류기 항상 학습\nfor param in model.classifier.parameters():\n    param.requires_grad = True\n\n'

In [ ]:
def transform(data):
    return tokenizer(
        data["document"],
        truncation=True,
        max_length=256,
        return_token_type_ids=False
    )

hf_dataset = hf_dataset.map(transform, batched=True)

hf_train_dataset = hf_dataset["train"]
hf_val_dataset = hf_dataset["validation"]
hf_test_dataset = hf_dataset["test"]


In [75]:
%pip install "transformers[torch]"
%pip install accelerate

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import numpy as np
from transformers import Trainer, TrainingArguments

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:


output_dir = 'transformers'
'''
training_arguments = TrainingArguments(
    output_dir,                                         # output이 저장될 경로
    eval_strategy="epoch",           #evaluation하는 빈도
    learning_rate = 2e-5,                         #learning_rate
    per_device_train_batch_size = 32,   # 각 device 당 batch size
    per_device_eval_batch_size = 32,    # evaluation 시에 batch size
    num_train_epochs = 3,                     # train 시킬 총 epochs
    weight_decay = 0.01,                        # weight decay
        fp16=True,
)
'''

training_arguments = TrainingArguments(
    output_dir="output",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    num_train_epochs=3,
    weight_decay=0.01,

    bf16=True,              
    tf32=True,              
    gradient_accumulation_steps=1,

    dataloader_num_workers=8,
    dataloader_pin_memory=True,
    report_to="none",

    logging_steps=10,
    save_strategy="epoch",
)



In [77]:
%pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [78]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [79]:
from evaluate import load
metric = load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return metric.compute(predictions=preds, references=labels)

In [80]:
model.to("cuda")

import torch
device = torch.device("cuda")
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, max_length=256, pad_to_multiple_of=16)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=hf_dataset["train"],
    eval_dataset=hf_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

trainer.train()

/tmp/ipykernel_4736/4158713387.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.228500,0.238342,0.901300
2,0.178200,0.240567,0.905080
3,0.129800,0.268323,0.905600


TrainOutput(global_step=3165, training_loss=0.1978018501735223, metrics={'train_runtime': 723.9829, 'train_samples_per_second': 559.405, 'train_steps_per_second': 4.372, 'total_flos': 1.065599774208e+17, 'train_loss': 0.1978018501735223, 'epoch': 3.0})

In [81]:
results = trainer.evaluate(hf_test_dataset)
print(results)

{'eval_loss': 0.26832345128059387, 'eval_accuracy': 0.9056, 'eval_runtime': 22.2519, 'eval_samples_per_second': 2247.004, 'eval_steps_per_second': 17.572, 'epoch': 3.0}


In [82]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.26832345128059387, 'eval_accuracy': 0.9056, 'eval_runtime': 22.5801, 'eval_samples_per_second': 2214.337, 'eval_steps_per_second': 17.316, 'epoch': 3.0}
